In [ ]:
# Step 1: Import Libraries
import pandas as pd
import xml.etree.ElementTree as ET
import os
import re
import xmltodict
from datetime import datetime

def xml_to_dict(element):
    """Recursively converts an XML element and its children to a dictionary."""
    if len(element) == 0:
        return element.text
    return {child.tag: xml_to_dict(child) for child in element}

def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))

In [ ]:
# Step 2: Load and Parse XML Data using pandas.read_xml
def load_performance_data(file_path):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()
    print("Loading PP data for: ", os.path.basename(file_path))

    # Extract date from filename
    date_match = re.search(r'\d{8}', os.path.basename(file_path))
    race_date = ""
    if date_match:
        race_date = date_match.group(0)

    # Extract track from filename
    track_name = os.path.basename(file_path).split('_')[0][-3:]
    
    # Extract each Race element within EntryRaceCard and convert to a dictionary
    races = []
    for race in root.findall('.//Race'):
        race_dict = xmltodict.parse(ET.tostring(race))['Race']
        if race_dict['BreedType']['Value'] != 'TB':
            continue
        race_dict = extract_general_race_info(race_dict, race_date, track_name)

        for entry in race.findall('.//Starters'):
            entry_dict = extract_entry_info(entry)
            workout_dict = extract_workout_info(entry)
            races.append({**race_dict, **entry_dict, **workout_dict})
        
    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(races)

    return df

def extract_general_race_info(race_dict, race_date, track_name):
    # Extract date from filename
    race_id = f"{race_date}_{race_dict['RaceNumber']}_{track_name}"
    race_date = race_date
    return {
        "race_id": race_id,
        "course_type": str(race_dict['Course']['Surface']['Value']),
        "distance": int(race_dict['Distance']['DistanceId']),
        "race_type": str(race_dict['RaceType']['RaceType']),
        "restriction_type": str(race_dict['RestrictionType']['Value']),
        "condition": str(race_dict['ConditionsOfRace']).strip(),
        "purse": float(race_dict['PurseUSA']),
        "number_of_run": int(race_dict['NumberOfRunners'])
    }

def extract_entry_info(entry_root):
    entry_dict = xmltodict.parse(ET.tostring(entry_root))['Starters']

    # Convert odds from fraction to decimal
    odds_fraction = entry_dict['Odds']
    if isinstance(odds_fraction, str) and '/' in odds_fraction:
        numerator, denominator = map(float, odds_fraction.split('/'))
        odds_decimal = numerator / denominator
    else:
        odds_decimal = float(odds_fraction) if odds_fraction else None
    
    final_dict = {
        "horse_id": f"{entry_dict['Horse']['HorseName']}_{entry_dict['ProgramNumber']}",
        "gender": str(entry_dict['Horse']['Sex']['Value']),
        "post_position": int(entry_dict['PostPosition']),
        "weight": int(entry_dict['WeightCarried']),
        "equipment": str(entry_dict['Equipment']['Value']),
        "medication": str(entry_dict['Medication']['Value']),
        "trainer": int(entry_dict['Trainer']['ExternalPartyId']),
        "jockey": int(entry_dict['Jockey']['ExternalPartyId']),
        "odds": float(odds_decimal),
    }


    for i, pp in enumerate(entry_root.findall('.//PastPerformance')):
        pp_dict = xmltodict.parse(ET.tostring(pp))['PastPerformance']
        final_dict.update({
            f"pp_track_{i}": str(pp_dict['Track']['TrackID']),
            f"pp_date_{i}": datetime.strptime(pp_dict['RaceDate'][:10], '%Y-%m-%d'),
            f"pp_course_type_{i}": str(pp_dict['Course']['Surface']['Value']),
            f"pp_distance_{i}": int(pp_dict['Distance']['DistanceId']),
            f"pp_race_type_{i}": str(pp_dict['RaceType']['RaceType']),
            f"pp_restriction_type_{i}": str(pp_dict['RaceRestrictions']['RestrictionType']),
            f"pp_condition_{i}": str(pp_dict['ConditionsOfRace']).strip(),
            f"pp_purse_{i}": float(pp_dict['PurseUSA']),
            f"pp_number_of_run_{i}": int(pp_dict['NumberOfStarters']),
            f"pp_finish_position_{i}": int(pp_dict['Start']['OfficialFinish']),
            f"pp_class_rating_{i}": int(pp_dict['Start']['ClassRating']),
            f"pp_speed_rating_{i}": int(pp_dict['Start']['SpeedFigure']),
        })


    return final_dict

def extract_workout_info(entry_root):
    final_dict = {}
    for i, workout in enumerate(entry_root.findall('.//Workout')):
        workout_dict = xmltodict.parse(ET.tostring(workout))['Workout']
        final_dict.update({
            f"workout_date_{i}": datetime.strptime(workout_dict['Date'][:10], '%Y-%m-%d'),
            f"workout_distance_{i}": int(workout_dict['Distance']['DistanceId']),
            f"workout_course_type_{i}": str(workout_dict['CourseType']['Surface']['Value']),
            f"workout_time_{i}": int(workout_dict['Timing']),
            f"workout_rank_{i}": int(workout_dict['Ranking']) / int(workout_dict['NumberInRankingGroup']),
        })

    return final_dict


performance_path = "C:\\Users\\dylan\\OneDrive - Wayne State College\\Documents\\Equibase Data 2023\\2023 PPs\\Extracted PPs"
file_suffixes = ['PRM_USA.xml', 'CBY_USA.xml', 'FON_USA.xml']  # Add your suffixes here

# Load all past performance files
performance_data = pd.concat([load_performance_data(os.path.join(root, file)) 
                              for root, _, files in os.walk(performance_path) 
                              for file in files if file.endswith(tuple(file_suffixes))])

In [ ]:
def load_results_data(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    print("Loading res data for: ", os.path.basename(file_path))

    entries = []
    for race in root.findall('.//RACE'):
        race_date = re.search(r'\d{8}', os.path.basename(file_path)).group(0)
        race_number = race.get("NUMBER")
        track_name = os.path.basename(file_path).split('_')[0][:3].upper()

        for entry in race.findall('.//ENTRY'):
            horse_name = entry.find(".//NAME").text
            horse_number = entry.find(".//PROGRAM_NUM").text
            entry_data = {
                "horse_id": f"{horse_name}_{horse_number}",
                "race_id": f"{race_date}_{race_number}_{track_name}",
                "Position": int(entry.find(".//POINT_OF_CALL[@WHICH='FINAL']").find('.//POSITION').text),
            }
            entries.append(entry_data)
    
    return entries

results_path = "C:\\Users\\dylan\\OneDrive - Wayne State College\\Documents\\Equibase Data 2023\\2023 Result Charts"
file_prefixes = ['prm', 'cby', 'fon']  # Add your suffixes here


# Load all results files
all_races = []
for file in os.listdir(results_path):
    if file.endswith('.xml') and file.startswith(tuple(file_prefixes)):
        file_path = os.path.join(results_path, file)
        all_races.extend(load_results_data(file_path))

results_data = pd.DataFrame(all_races)

In [ ]:
# Step 3: Combine Data

# Merge the DataFrames on RaceNumber and race_date
merged_data = pd.merge(performance_data, results_data, on=["race_id", "horse_id"], how='inner')

# Clean up old DataFrames
# del performance_data
# del results_data

In [ ]:
# Check for any issues in the merged data
print(merged_data.head())
print(merged_data.info())

# Select features and target variable
features = merged_data.drop(columns=['Position'])
target = merged_data['Position']

# Verify the target variable
print(target.describe())

# Identify datetime features
datetime_features = features.select_dtypes(include=['datetime64']).columns

# Convert datetime features to numeric format
for feature in datetime_features:
    features[feature + '_year'] = features[feature].dt.year
    features[feature + '_month'] = features[feature].dt.month
    features[feature + '_day'] = features[feature].dt.day
    features = features.drop(columns=[feature])

# Identify categorical features
categorical_features = features.select_dtypes(include=['object']).columns

# Apply one-hot encoding
features = pd.get_dummies(features, columns=categorical_features, drop_first=True)

# Handle missing values by filling them with the mean of the column
features = features.fillna(features.mean())

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

# Create an imputer instance to fill missing values
imputer = SimpleImputer(strategy='mean')  # You can change the strategy as needed

# Group past performances
for i in range(10):
    # get all columns beginning with pp and ending with i not containing interaction
    columns_to_group = [col for col in features.columns if col.startswith('pp') and col.endswith(str(i)) and not col.startswith('pp_interaction')]

    # Impute missing values
    imputed_data = imputer.fit_transform(merged_data[columns_to_group])

    # Create interaction terms
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    interaction_terms = poly.fit_transform(imputed_data)

    # Optionally, apply PCA to reduce dimensionality
    pca = PCA(n_components=2)  # Adjust the number of components as needed
    reduced_features = pca.fit_transform(interaction_terms)

    for j in range(reduced_features.shape[1]):
        features[f'pp_interaction_{i}_{j}'] = reduced_features[:, j]

    merged_data = merged_data.drop(columns=columns_to_group)

# Group workouts
for i in range(4):
    # get all columns beginning with workout and ending with i
    columns_to_group = [col for col in features.columns if col.startswith('workout') and col.endswith(str(i)) and not col.startswith('workout_interaction')]

    # Impute missing values
    imputed_data = imputer.fit_transform(merged_data[columns_to_group])

    # Create interaction terms
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    interaction_terms = poly.fit_transform(imputed_data)

    # Optionally, apply PCA to reduce dimensionality
    pca = PCA(n_components=2)  # Adjust the number of components as needed
    reduced_features = pca.fit_transform(interaction_terms)

    for j in range(reduced_features.shape[1]):
        features[f'workout_interaction_{i}_{j}'] = reduced_features[:, j]

    merged_data = merged_data.drop(columns=columns_to_group)

In [ ]:
from sklearn.model_selection import train_test_split

# Step 5: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', LinearSVC(max_iter=10000))
])

# Define the search space
search_space = {
    'svc__C': Real(1e-6, 1e+6, prior='log-uniform'),
    'svc__tol': Real(1e-6, 1e-1, prior='log-uniform')
}

# Initialize BayesSearchCV
opt = BayesSearchCV(
    estimator=pipeline,
    search_spaces=search_space,
    n_iter=32,
    cv=5,
    n_jobs=-1,
    verbose=0
)

# Perform the optimization
opt.fit(X_train, y_train)

# Best parameters found
print("Best parameters found: ", opt.best_params_)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Step 7: Evaluate the model
y_pred = opt.predict(X_test)

In [ ]:
import numpy as np

# Step 8: Clip predictions to be within the range [1, number_of_run]
number_of_run = X_test['number_of_run'].values  # Assuming 'number_of_run' is a column in X_test
clipped_predictions = np.clip(y_pred, 1, number_of_run)

In [ ]:
mse = mean_squared_error(y_test, clipped_predictions)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, clipped_predictions)
print(f'R² Score: {r2}')

In [ ]:
import matplotlib.pyplot as plt
# Assuming 'odds' column is present in the test set
odds = X_test['odds']

# Step 8: Visualize the results
plt.figure(figsize=(10, 6))
plt.scatter(y_test, clipped_predictions, alpha=0.5, s=odds*10)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title('True Values vs Predictions')
plt.show()

In [ ]:
# combine y_test and y_pred into a single DataFrame
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Odds': odds})